In [7]:
import pandas as pd
import numpy as np
from numpy import arctan as atan, sqrt, sin, cos

### Augmented measures

The original sensor data $a_f$, $a_l$, $a_v$ (front, vertical, and lateral sensors, respectively) are augmented messures as described in Wickramasinghe et al (2017, appendix B). Firstly angles $\theta$, $\alpha$, and $\beta$ are defined as follows:


$\theta \approx \tan^{-1}(a_f / \sqrt{a_v^2 + a_l ^2})$

$\alpha \approx \tan^{-1}(a_l / a_v)$

$\beta \approx \tan^{-1}(a_l / a_f)$


Then approximate acceleration signals $a_x$, $a_y$, and $a_z$ are derived as shown below:

$a_x \approx a_v\sin(\theta) + a_f\cos(\theta)$

$a_y \approx a_v\sin(\alpha) + a_l\cos(\alpha)$

$a_z \approx 1 + a_v\cos(\theta)\cos(\alpha) + a_l\sin(\alpha) + a_f\sin(\theta)$

### Features

Features are taken over a window $X_{[t_i - \delta t, t_i]}$:

| Feature | Description |
|:-:|:-|
| $a_f$ | Frontal acceleration |
| $a_l$ | Lateral acceleration |
| $a_v$ | Vertical acceleration |
| $\theta$ | Angle on Sagittal plane |
| $\alpha$ | Angle on Coronal plane |
| $\beta$ | Angle on Transverse plane |
| $a_x$ | Anteroposterior acceleration |
| $a_y$ | Mediolateral acceleration |
| $a_z$ | Dorsvental acceleration |
| $\mbox{mean}$$(a_f)$; $\mbox{mean}$$(a_l)$; $\mbox{mean}$$(a_v)$ |
| $\mbox{max}$$(a_f)$; $\mbox{max}$$(a_l)$; $\mbox{max}$$(a_v)$ |
| $\mbox{min}$$(a_f)$; $\mbox{min}$$(a_l)$; $\mbox{min}$$(a_v)$ |
| $\mbox{corr}$$(a_f\mbox{,}a_l)$; $\mbox{corr}$$(a_f\mbox{,}a_v)$; $\mbox{corr}$$(a_l\mbox{,}a_v)$ |
| $\mbox{mean}$ $(a_x)$; $\mbox{mean}$ $(a_y)$; $\mbox{mean}$ $(a_z)$ | 
| $\mbox{mean}$$(\theta)$; $\mbox{mean}$$(\alpha)$; $\mbox{mean}$$(\beta)$ |
| $\mbox{max}$$(\theta)$; $\mbox{max}$$(\alpha)$; $\mbox{max}$$(\beta)$ |
| $\mbox{min}$$(\theta)$; $\mbox{min}$$(\alpha)$; $\mbox{min}$$(\beta)$ |

In [3]:
from pathlib import Path

import pandas as pd

DATA_ROOT = Path('..') / 'data'

dfs = []
activity_labels = ['bed', 'chair', 'lying', 'ambulating']
default_names = ['time', 'front', 'vertical', 'lateral', 'sensor_id', 'rssi', 'phase', 'frequency', 'activity']
for data_file in Path(DATA_ROOT).rglob('d[12]p??[FM]'):
    df = pd.read_csv(data_file, names=default_names)
    df['activity_label'] = df['activity'].apply(lambda i: activity_labels[i - 1])
    df['gender_label'] = str(data_file)[-1]
    df['participant'] = data_file.name
    
    # Add a column indicating order of the activities for a particiapnt.
    df = df.sort_values(by=['time'])
    df['activity_sequence'] = (df['activity'].shift(1) != df['activity']).cumsum()
    dfs.append(df)

sensor_df = pd.concat(dfs, axis='index')
sensor_df = sensor_df.sort_values(by=['participant', 'time'])

sensor_df.head()

,time,front,vertical,lateral,sensor_id,rssi,phase,frequency,activity,activity_label,gender_label,participant,activity_sequence
0,0.0,0.27203,1.00820,-0.082102,1,-63.5,2.4252,924.25,1,bed,M,d1p01M,1
0,0.0,0.27203,1.00820,-0.082102,1,-63.5,2.4252,924.25,1,bed,M,d1p01M,1
1,0.5,0.27203,1.00820,-0.082102,1,-63.0,4.7369,921.75,1,bed,M,d1p01M,1
1,0.5,0.27203,1.00820,-0.082102,1,-63.0,4.7369,921.75,1,bed,M,d1p01M,1
2,1.5,0.44791,0.91636,-0.013684,1,-63.5,3.0311,923.75,1,bed,M,d1p01M,1


In [9]:

t = sensor_df['time']
a_f = sensor_df['front']
a_l = sensor_df['lateral']
a_v = sensor_df['vertical']

theta = atan(a_f / sqrt(a_v**2 + a_l**2))
alpha = atan(a_l / a_v)
beta = atan(a_l / a_f)

a_x = a_v * sin(theta) + a_f * cos(theta)
a_y = a_v * sin(alpha) + a_l * cos(alpha)
a_z = 1 + a_v * cos(theta) * cos(alpha) + a_l * sin(alpha) + a_f * sin(theta)
